In [ ]:
# baseline

#  GP
#  S-ARIMA
#  SVR


#  RF
#  GBT
#  xgboosted 


# bayeisan regression

# LSTM:
#   1. initialization
#   2. batch normalization
#   3. weight normalization
#   4. variable length
#   5. attention mechanism 

# LSTM:  discrimitive, generative
# perodic in data
# attention 

# http://bugra.github.io/work/notes/2014-04-26/outlier-detection-markov-chain-monte-carlo-via-pymc/

In [ ]:
# TO DO:
#  lstm regularization
#  skewness of dependent variable 

# http://smerity.com/articles/2016/orthogonal_init.html



# RMSE, MAE, MAPE 

In [1]:
# data processing packages
import numpy as np   
import pandas as pd 
from pandas import *
from numpy import *

from scipy import stats # look at scipy
from scipy import linalg
from scipy import *
 
%matplotlib inline    
import matplotlib as mplt
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import IPython
from IPython.display import display

import matplotlib
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
# matplotlib.style.use('ggplot')
import itertools

import random

# machine leanring packages

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import xgboost as xgb

import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist

from utils_keras import *
from utils_dataPrepro import *


Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [ ]:
# features:
    
# Feature based approach: Here the time series are mapped to another, 
#     possibly lower dimensional, representation. 
#     This means that the feature extraction algorithm calculates characteristics
#     such as the average or maximal value of the time series. The features are then 
#     passed as a feature matrix to a "normal" machine learning such as a neural network, 
#     random forest or support vector machine. This approach has the advantage of a better
#     explainability of the results. Further it enables us to use a well developed theory of
# supervised machine learning.


In [18]:
# stock data

files_list=["../dataset/dataset_ts/stock_xtrain.dat", \
            "../dataset/dataset_ts/stock_xtest.dat",\
            "../dataset/dataset_ts/stock_ytrain.dat", \
            "../dataset/dataset_ts/stock_ytest.dat"]

xtrain_df = pd.DataFrame( flatten_features(np.load(files_list[0])) )
xtest_df  = pd.DataFrame( flatten_features(np.load(files_list[1])) )
ytrain_df = pd.DataFrame( flatten_features(np.load(files_list[2])) )
ytest_df  = pd.DataFrame( flatten_features(np.load(files_list[3])) )

xtrain, ytrain, xtest, ytest = prepare_train_test_data(200, True, \
                                                       xtrain_df, xtest_df, ytrain_df, ytest_df)
    
    
xtrain = flatten_features( xtrain )
ytrain = flatten_features( ytrain )
xtest  = flatten_features( xtest )
ytest  = flatten_features( ytest )

print np.shape(xtrain), np.shape(ytrain), np.shape(xtest), np.shape(ytest)

(3800, 398) (3800, 1) (1300, 398) (1300, 1)


In [3]:
# power data

files_list=["../dataset/dataset_ts/power_xtrain.dat", \
            "../dataset/dataset_ts/power_xtest.dat",\
            "../dataset/dataset_ts/power_ytrain.dat", \
            "../dataset/dataset_ts/power_ytest.dat"]

xtrain_df = pd.DataFrame( flatten_features(np.load(files_list[0])) )
xtest_df  = pd.DataFrame( flatten_features(np.load(files_list[1])) )
ytrain_df = pd.DataFrame( flatten_features(np.load(files_list[2])) )
ytest_df  = pd.DataFrame( flatten_features(np.load(files_list[3])) )

xtrain, ytrain, xtest, ytest = prepare_train_test_data(200, True, \
                                                       xtrain_df, xtest_df, ytrain_df, ytest_df)

xtrain = flatten_features( xtrain )
ytrain = flatten_features( ytrain )
xtest  = flatten_features( xtest )
ytest  = flatten_features( ytest )
    
print np.shape(xtrain), np.shape(ytrain), np.shape(xtest), np.shape(ytest)

(8300, 398) (8300, 1) (1297, 398) (1297, 1)


In [4]:
# GBT

# https://www.analyticsvidhya.com/blog/2016/02/
# complete-guide-parameter-tuning-gradient-boosting-gbm-python/

#----Boosting parameters:
#   learnning rate: 0.05 - 0.2
#   n_estimators: 40-70

#----Tree parameters:
#   max_depth: 3-10
#   max_leaf_nodes
#   num_samples_split: 0.5-1% of total number 
#   min_samples_leaf
#   max_features

#   subsample: 0.8
#   min_weight_fraction_leaf

#----Order of tuning: max_depth and num_samples_split, min_samples_leaf, max_features

def gbt_n_estimatior(maxnum, X, Y, xtest, ytest, fix_lr ):
    
    tmpy = Y.reshape( (len(Y),) )
    score = []
    
    cnt = len(xtest)
    
    for i in range(10,maxnum+1,10):
        clf = GradientBoostingRegressor(n_estimators = i,learning_rate = fix_lr)
        
        clf.fit( X, tmpy )
        
        pytest = clf.predict(xtest)

        score.append(\
        (i, sqrt(mean([( pytest[i]-ytest[i] )**2 for i in range(cnt) ]))) )
    
    return min(score, key = lambda x: x[1]), score


def gbt_tree_para( X, Y, xtest, ytest, depth_range, fix_lr, fix_n_est):
    
    tmpy = Y.reshape( (len(Y),) )
    score = []
    
    cnt = len(xtest)
    
    for i in depth_range:
        
        clf = GradientBoostingRegressor(n_estimators = fix_n_est,learning_rate = fix_lr,\
                                        max_depth = i )
        
        clf.fit( X, tmpy )
        
        pytest = clf.predict(xtest)

        score.append(\
        (i, sqrt(mean([( pytest[i]-ytest[i] )**2 for i in range(cnt) ]))) )
    
    return min(score, key = lambda x: x[1]), score
        
    

In [7]:
# GBT performance

fix_lr = 0.25

n_err, n_err_list = gbt_n_estimatior(301, xtrain, ytrain, xtest, ytest, fix_lr)

print "n_estimator, RMSE:", n_err

depth_err, depth_err_list = gbt_tree_para( xtrain, ytrain, xtest, ytest, range(3,16), fix_lr, n_err[0] )

print "depth, RMSE:", depth_err


n_estimator, RMSE: (40, 1.0143259393877819)
depth, RMSE: (3, 1.014234534209864)


In [11]:
# # GBT test
# clf = GradientBoostingRegressor(n_estimators=20, learning_rate=0.25, max_depth = 3)
        
# clf.fit( xtrain, ytrain )
        
# pytest = clf.predict(xtest)

# err = []
# for i in range(len(pytest)):
#     err.append( ytest[i] - pytest[i] )
    

# print mean(err), var(err), sqrt(mean([i**2 for i in err]))

# print zip(ytrain, pytest)

In [5]:
# XGBoosted

# https://www.analyticsvidhya.com/blog/2016/03/
#     complete-guide-parameter-tuning-xgboost-with-codes-python/

#----General Parameters

#   eta(learning rate): 0.05 - 0.3
#   number of rounds: 

#----Booster Parameters

#   max_depth 3-10
#   max_leaf_nodes
#   gamma: mininum loss reduction
#   min_child_weight: 1 by default

#   max_delta_step: not needed in general, for unbalance in logistic regression
#   subsample: 0.5-1
#   colsample_bytree: 0.5-1
#   colsample_bylevel: 
#   lambda: l2 regularization 
#   alpha: l1 regularization
#   scale_pos_weight: >>1, for high class imbalance

# Learning Task Parameters


def xgt_n_depth( lr, max_depth, max_round, X, Y, xtest, ytest ):
    
    score = []
    xg_train = xgb.DMatrix(xtrain, label = ytrain)
    xg_test  = xgb.DMatrix(xtest,  label = ytest)

# setup parameters for xgboost
    param = {}
# use softmax multi-class classification
    param['objective'] = "reg:linear" 
#   'multi:softmax'
    
# scale weight of positive examples
    param['eta'] = lr
    param['max_depth'] = 0
    param['silent'] = 1
    param['nthread'] = 8
#     param['num_class'] = 8
#     param['gamma']
    
    for depth_trial in range(2, max_depth):
        for num_round_trial in range(2, max_round):
            
            param['max_depth'] = depth_trial
            bst  = xgb.train( param, xg_train, num_round_trial )
            pred = bst.predict( xg_test )
            
            tmp_accur = sqrt(mean( [(pred[i] - ytest[i])**2 for i in range(len(ytest))] )) 
            
            score.append( (depth_trial, num_round_trial, tmp_accur) )
            
    return min(score, key = lambda x: x[2]), score


def xgt_l2( fix_lr, fix_depth, fix_round, X, Y, xtest, ytest, l2_range ):
    
    score = []
    xg_train = xgb.DMatrix(xtrain, label = ytrain)
    xg_test  = xgb.DMatrix(xtest,  label = ytest)

# setup parameters for xgboost
    param = {}
# use softmax multi-class classification
    param['objective'] = 'reg:linear' 
#   'multi:softmax'
    
# scale weight of positive examples
    param['eta'] = fix_lr
    param['max_depth'] = fix_depth
    param['silent'] = 1
    param['nthread'] = 8
#     param['num_class'] = 8
    
    param['lambda'] = 0.0
#     param['alpha']
    
    
    for l2_trial in l2_range:
        
        param['lambda'] = l2_trial
        
        bst = xgb.train(param, xg_train, fix_round )
        pred = bst.predict( xg_test )
            
        tmp_accur = sqrt(mean( [(pred[i] - ytest[i])**2 for i in range(len(ytest))] )) 
            
        score.append( (l2_trial, tmp_accur) )
            
    return min(score, key = lambda x: x[1]), score

    
#  def xgt_l1 for very high dimensional features    
    

In [8]:
# XGBoosted performance
fix_lr = 0.2

n_depth_err, n_depth_err_list = xgt_n_depth( fix_lr, 16, 51, xtrain, ytrain, xtest, ytest)

print " depth, number of rounds, RMSE:", n_depth_err

l2_err, l2_err_list = xgt_l2( fix_lr, n_depth_err[0], n_depth_err[1], xtrain, ytrain, xtest, ytest,\
                    [0.0001, 0.001, 0.01, 0.1, 1, 10, 100])

print " l2, RMSE:", l2_err

 depth, number of rounds, RMSE: (3, 44, 1.0455573756115384)
 l2, RMSE: (0.01, 1.0192915193168073)


In [6]:
# Random forest

# max_features:
# n_estimators
# max_depth

def rf_n_depth_estimatior(maxnum, maxdep, X, Y, xtest, ytest ):
    
    tmpy = Y.reshape( (len(Y),) )
    score = []
    
    cnt = len(xtest)
        
    for n_trial in range(10,maxnum+1,10):
        for dep_trial in range(2, maxdep+1):
            
            clf = RandomForestRegressor(n_estimators = n_trial, max_depth = dep_trial, max_features = "sqrt")
            clf.fit( X, tmpy )
        
            pytest = clf.predict(xtest)
            
            score.append(\
            (n_trial, dep_trial, sqrt(mean([( pytest[i]-ytest[i] )**2 for i in range(cnt) ]))) )
    
    return min(score, key = lambda x: x[2]), score


In [9]:
# Random forest performance

n_err, n_err_list = rf_n_depth_estimatior( 100, 25, xtrain, ytrain, xtest, ytest )

print "n_estimator, RMSE:", n_err


n_estimator, RMSE: (90, 13, 1.2387303782002563)
